In [23]:
# Date: 4/4/2019
# Author: Brandon Lucas
# Research Professor: Dr. Ian McDonough

# what modules do we need?
import os
import sys
import tensorflow as tf
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.models import Sequential
from keras import layers
from keras import models
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from organize_melodic_functions import *

In [24]:
orig_dir = "C:/Users/thebr/Desktop/Programming/melodic-classifier/testing_data"
data_dir = "C:/Users/thebr/Desktop/Programming/melodic-classifier/testing_data"

In [25]:
try:
    os.chdir(orig_dir)
except:
    print('Could not change to that directory')
    sys.exit(0)

artifact_images = []
non_artifact_images = []
count = 0
for root, dirs, files in os.walk(data_dir):
    for directory in dirs:
        # make this an option later
        if "Preclin" in directory:
            os.chdir(directory)
            # the files that indicate which id's are artifacts
            artifact_numbers_file = open("fix_s8.txt")
            line = artifact_numbers_file.readline()
            artifact_numbers_file.close()
            artifact_numbers = line.strip().split(",")
            # every var after "t" is substrings to exclude
#             thresh_files = getFiles(root + '/' + directory, "t", "EV", "index", "IC", "._", "f", ".html", "DS", "txt")
            thresh_files = getFiles(root + '/' + directory, "thresh", "._")
            # loop through the thresh files
            for file_path in thresh_files:
                filename = file_path.split("/")[-1]
#                 id = filename[filename.rindex('t')+1:].split(".")[0]
                id = filename.split("_")[1]
                if id in artifact_numbers:
                    artifact_images.append(file_path)
                else:
                    non_artifact_images.append(file_path)
        os.chdir(root)

# image = Image.open(artifact_images[0])
# image = image.resize((238, 1000))
# image.show()

In [26]:
data = []
labels = []
for artifact_image in artifact_images:
    image = mpimg.imread(artifact_image)
    image_from_array = Image.fromarray(image, 'RGB')
#     image = Image.open(artifact_image)
    size_image = image_from_array.resize((238, 1000))
#     size_image = image_from_array
#     image = image.resize((992, 4167))
#     data.append(np.array(image))
    data.append(np.array(size_image))
    labels.append(1)
    
for non_artifact_image in non_artifact_images:
#     image = mpimg.imread(non_artifact_image)
#     image_from_array = Image.fromarray(image, 'RGB')
#     size_image = image_from_array.resize((992, 4167))
#     image = Image.open(artifact_image)
#     image = image.resize((992, 4167))
#     data.append(np.array(image))
    image = mpimg.imread(artifact_image)
    image_from_array = Image.fromarray(image, 'RGB')
#     image = Image.open(artifact_image)
    size_image = image_from_array.resize((238, 1000))
#     image_from_array = Image.fromarray(image, 'RGB').crop(0, height/2, 0, 0)
#     image = image.resize((992, 4167))
#     data.append(np.array(image))
    data.append(np.array(size_image))
#     data.append(np.array(size_image))
    labels.append(0)

In [27]:
mri = np.array(data)
labels = np.array(labels)
s = np.arange(mri.shape[0])
np.random.shuffle(s)
mri = mri[s]
labels = labels[s]

# (X_train, X_test) = df[(int)(0.1*len(df)):],df[:(int)(0.1*len(df))]
# (y_train, y_test) = labels[(int)(0.1*len(labels)):],labels[:(int)(0.1*len(labels))]
# X_train.shape

In [28]:
(X_train, X_test) = mri[(int)(0.1*len(mri)):], mri[:(int)(0.1*len(mri))]
# X_train = X_train.astype('float32') / 255 # normalize data by 255 for images
# X_test = X_test.astype('float32') / 255
train_len = len(X_train)
test_len = len(X_test)

(y_train, y_test) = labels[(int)(0.1*len(labels)):],labels[:(int)(0.1*len(labels))]
X_train.shape

(558, 1000, 238, 3)

In [29]:
# from keras.applications.inception_v3 import InceptionV3
# from keras.layers import Input

# input_tensor = Input(shape=(238, 1000, 3))  # this assumes K.image_data_format() == 'channels_last'
# model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

In [72]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(1000, 238, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(1000, 238, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu",input_shape=(1000, 238, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(250, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2,activation="softmax"))#2 represent output layer neurons
model.summary()
# img_height = 1000
# img_width = 238
# img_channels = 3

# #
# # network params
# #

# cardinality = 1


# def residual_network(x):
#     """
#     ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
#     """
#     def add_common_layers(y):
#         y = layers.BatchNormalization()(y)
#         y = layers.LeakyReLU()(y)

#         return y

#     def grouped_convolution(y, nb_channels, _strides):
#         # when `cardinality` == 1 this is just a standard convolution
#         if cardinality == 1:
#             return layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
        
#         assert not nb_channels % cardinality
#         _d = nb_channels // cardinality

#         # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
#         # and convolutions are separately performed within each group
#         groups = []
#         for j in range(cardinality):
#             group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
#             groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
            
#         # the grouped convolutional layer concatenates them as the outputs of the layer
#         y = layers.concatenate(groups)

#         return y

#     def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False):
#         """
#         Our network consists of a stack of residual blocks. These blocks have the same topology,
#         and are subject to two simple rules:
#         - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
#         - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
#         """
#         shortcut = y

#         # we modify the residual building block as a bottleneck design to make the network more economical
#         y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
#         y = add_common_layers(y)

#         # ResNeXt (identical to ResNet when `cardinality` == 1)
#         y = grouped_convolution(y, nb_channels_in, _strides=_strides)
#         y = add_common_layers(y)

#         y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
#         # batch normalization is employed after aggregating the transformations and before adding to the shortcut
#         y = layers.BatchNormalization()(y)

#         # identity shortcuts used directly when the input and output are of the same dimensions
#         if _project_shortcut or _strides != (1, 1):
#             # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
#             # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
#             shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
#             shortcut = layers.BatchNormalization()(shortcut)

#         y = layers.add([shortcut, y])

#         # relu is performed right after each batch normalization,
#         # expect for the output of the block where relu is performed after the adding to the shortcut
#         y = layers.LeakyReLU()(y)

#         return y

#     # conv1
#     x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
#     x = add_common_layers(x)

#     # conv2
#     x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
#     for i in range(3):
#         project_shortcut = True if i == 0 else False
#         x = residual_block(x, 128, 256, _project_shortcut=project_shortcut)

#     # conv3
#     for i in range(4):
#         # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
#         strides = (2, 2) if i == 0 else (1, 1)
#         x = residual_block(x, 256, 512, _strides=strides)

#     # conv4
#     for i in range(6):
#         strides = (2, 2) if i == 0 else (1, 1)
#         x = residual_block(x, 512, 1024, _strides=strides)

#     # conv5
#     for i in range(3):
#         strides = (2, 2) if i == 0 else (1, 1)
#         x = residual_block(x, 1024, 2048, _strides=strides)

#     x = layers.GlobalAveragePooling2D()(x)
#     x = layers.Dense(1)(x)

#     return x


# image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
# network_output = residual_network(image_tensor)
  
# model = models.Model(inputs=[image_tensor], outputs=[network_output])
# print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_47 (Conv2D)           (None, 1000, 238, 16)     208       
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 500, 119, 16)      0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 500, 119, 16)      0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 500, 119, 16)      1040      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 250, 59, 16)       0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 250, 59, 16)       0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 250, 59, 64)       4160      
__________

In [73]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])


In [74]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor="loss", patience=10, verbose=0, mode="min")
mcp_save = ModelCheckpoint('model_weights.hdf5', save_best_only=True, monitor="loss", mode="min")
# reduce_lr_loss = ReduceLROnPlateu(monitor="val_loss", factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode="min")

In [75]:
model.fit(X_train,y_train, batch_size=15, epochs=50, callbacks=[early_stopping, mcp_save])

Epoch 1/50
558/558 [==============================] - 106s 191ms/step - loss: 6.1506 - acc: 0.6183
Epoch 2/50


C:\Users\thebr\Anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
C:\Users\thebr\Anaconda3\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


558/558 [==============================] - 102s 183ms/step - loss: 6.1237 - acc: 0.6201
Epoch 3/50
558/558 [==============================] - 102s 184ms/step - loss: 6.1237 - acc: 0.6201
Epoch 4/50
558/558 [==============================] - 103s 184ms/step - loss: 6.1237 - acc: 0.6201
Epoch 5/50
558/558 [==============================] - 103s 184ms/step - loss: 6.1237 - acc: 0.6201
Epoch 6/50
558/558 [==============================] - 103s 184ms/step - loss: 6.1237 - acc: 0.6201
Epoch 7/50
558/558 [==============================] - 103s 185ms/step - loss: 6.1237 - acc: 0.6201
Epoch 8/50
558/558 [==============================] - 102s 183ms/step - loss: 6.1237 - acc: 0.6201
Epoch 9/50
558/558 [==============================] - 101s 182ms/step - loss: 6.1237 - acc: 0.6201
Epoch 10/50
558/558 [==============================] - 103s 184ms/step - loss: 6.1237 - acc: 0.6201
Epoch 11/50
558/558 [==============================] - 103s 185ms/step - loss: 6.1237 - acc: 0.6201
Epoch 12/50
558/558

KeyboardInterrupt: 

In [46]:
X_loss, accuracy = model.evaluate(X_test,y_test)

print(accuracy)
print(X_loss)

InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 1 0 1 1 0 0 0 0 1 1 1 0 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 1
	 [[{{node loss_6/dense_12_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]